In [4]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics, preprocessing
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from catboost import Pool, CatBoostClassifier
import lightgbm as lgb


In [2]:
#Importing
train_data = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/training_v2.csv')
test_data =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/unlabeled.csv')

sub_data = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/samplesubmission.csv')
data_dict = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/WiDS Datathon 2020 Dictionary.csv')

In [3]:
def weighted_classt(x): 
    if pd.isna(x):
        return np.nan
    elif x < 15: 
        return 'very severely underweight' 
    elif x >= 15 and x < 16: 
        return 'severely weight' 
    elif x >=16 and x < 18.5: 
        return 'underweight' 
    elif x >= 18.5 and x < 25: 
        return 'healthy weight' 
    elif x >= 25 and x < 30: 
        return 'overweight'
    elif x >= 30 and x < 35: 
        return 'class 1' 
    elif x >= 35 and x < 40: 
        return 'class 2' 
    else: 
        return 'class 3' 

In [5]:
train_data['weightclass'] = train_data['bmi'].map(weighted_classt)
test_data['weightclass'] = test_data['bmi'].map(weighted_classt)

In [6]:
cat_var = list(data_dict[data_dict['Data Type'].isin(['binary','string','integer'])]['Variable Name'])
cat_var.remove('icu_admit_type')
cat_var.remove('bmi')
cat_var = cat_var[1:]

In [7]:
#cat_var = cat_var[1:]
#cat_var = cat_var[1:]
cat_var = cat_var + ['weightclass']

In [8]:
cont_var = list(data_dict[data_dict['Data Type'].isin(['numeric'])]['Variable Name'])
cont_var.remove('pred')
cont_var = cont_var + ['bmi']

#### Baseline

In [9]:
x_train = train_data.drop('hospital_death',axis=1)
y_train = train_data['hospital_death']

In [10]:
x_test = test_data.drop('hospital_death',axis=1)

In [11]:
x_comb = pd.concat([x_train,x_test],sort=False)

In [12]:
### impute bmi values based on height and weight
count = 0
bmi_list = []
for i in tqdm(range(x_comb.shape[0])):
    
    if pd.isnull(x_comb["weight"].iloc[i]) or pd.isnull(x_comb["height"].iloc[i]):
        bmi_list.append(x_comb["bmi"].iloc[i])
        continue
    else:
        
        count += 1
        bmi_list.append(x_comb["weight"].iloc[i] / ((x_comb["height"].iloc[i]/100) ** 2))

100%|██████████| 131021/131021 [00:06<00:00, 20221.92it/s]


In [13]:
x_comb['bmi'] = bmi_list

In [18]:
x_comb['apache_2_bodysystem'] = x_comb['apache_2_bodysystem'].str.replace('Undefined Diagnoses', 'Undefined diagnoses', regex=False)

In [20]:
x_comb["apache_4a_hospital_death_prob"] = x_comb["apache_4a_hospital_death_prob"].replace(-1, np.nan)
x_comb["apache_4a_icu_death_prob"] = x_comb["apache_4a_icu_death_prob"].replace(-1, np.nan)

In [21]:
new_cat_var = list(set(cat_var).intersection(set(x_comb.columns)))

In [22]:
for col in tqdm(new_cat_var):
    le = LabelEncoder()
    x_comb[col] = le.fit_transform(x_comb[col].astype(str))

100%|██████████| 32/32 [00:03<00:00,  8.56it/s]


In [23]:
new_cont_var = list(set(cont_var).intersection(set(x_comb.columns)))

In [24]:
x_comb["age"].mean()

62.00996491901006

In [ ]:
# for col in tqdm(new_cont_var):
#     x_comb[col] = x_comb[col].fillna(x_comb[col].mean())

In [25]:
count_var = []
for i,col in tqdm(enumerate(new_cat_var)):
    counter = Counter(x_comb[col])
    x_comb[str(col)+'count'] = x_comb[col].apply(lambda x:counter[x])
    count_var.append(str(col)+'count')

32it [00:01, 17.57it/s]


In [26]:
age_cat = []

for i in tqdm(range(0,len(x_comb))):
    
    val = x_comb['age'].iloc[i]
    
    if val >= 15 and val <= 24: 
        age_cat.append('igen')
    
    elif val >= 25 and val <= 54: 
       age_cat.append('Prime_working_Age')
    
    elif val >= 55 and val <= 64: 
        age_cat.append('Mature_working_Age')
        
    else: 
        age_cat.append('Elderly_working_Age')


100%|██████████| 131021/131021 [00:01<00:00, 73236.18it/s]


In [27]:
x_comb['age_category'] = age_cat
le = LabelEncoder()
x_comb['age_category'] = le.fit_transform(x_comb['age_category'].astype(str))

In [28]:
#List to be biined -----> VITALS
to_binned_columns = list(data_dict[data_dict['Category'] == 'APACHE covariate']['Variable Name'].unique())
to_binned_columns = list(set(to_binned_columns).intersection(set(x_comb.columns)))

In [29]:
#inter_cols3
#x_comb['d1_heartrate_max_cat'] = 

binned_cols = []

for cols in tqdm(to_binned_columns) :
    x_comb[cols+'_binned'] = list(pd.cut(x_comb[cols], 10,labels=False))
    binned_cols.append(cols+'_binned')
    

100%|██████████| 28/28 [00:00<00:00, 36.68it/s]


In [30]:
x_comb['height'+'_binned'] = list(pd.cut(x_comb['height'], 10,labels=False))

In [ ]:
#binned_cols

In [ ]:
#train_data.columns[0:100]

# list_imp_cols = ['icu_type','weightclass','age_category','gender','d1_heartrate_max_binned','height_binned']

# inter_cols4 = []

# for col1 in tqdm(list_imp_cols) :
#     for col2 in list_imp_cols:
#         if col1 !=col2 :
#             x_comb[col1+col2] = x_comb[col1].astype(str) + '_' + x_comb[col2].astype(str)
#             inter_cols4.append(col1+col2)
            

In [ ]:
# for cols in tqdm(inter_cols4):
#     le = LabelEncoder()
#     x_comb[cols] = le.fit_transform(x_comb[cols].astype(str))

In [31]:
#2way interaction
bias_cols = ['gender','age']

In [32]:
inter_cols6 = []

for i in tqdm(bias_cols):
    
    for j in binned_cols :
        
        x_comb[i+'_'+j] = x_comb[i].astype(str) + '_' + x_comb[j].astype(str)
        
        inter_cols6.append(i+'_'+j)

100%|██████████| 2/2 [00:07<00:00,  3.60s/it]


In [33]:
#Critical -3way interaction

# inter_cols6 = []

x_comb['age'+'gender'+'weighted_classt'] = x_comb['age'].astype(str) + '_' + x_comb['gender'].astype(str) + '_' + x_comb['weightclass'].astype(str)
x_comb['age'+'gender'+'ethnicity'] = x_comb['age'].astype(str) + '_' + x_comb['gender'].astype(str) + '_' + x_comb['ethnicity'].astype(str)

inter_cols6 = inter_cols6 + ['age'+'gender'+'weighted_classt'] +['age'+'gender'+'ethnicity']


In [34]:
for cols in tqdm(inter_cols6):
    le = LabelEncoder()
    x_comb[cols] = le.fit_transform(x_comb[cols].astype(str))

100%|██████████| 58/58 [00:02<00:00, 26.31it/s]


In [ ]:
#final_cat_var = new_cat_var + count_var + ['age_category'] + inter_cols1 + inter_cols2 + inter_cols3 + inter_cols4

##### Target Encoding

In [35]:
temp_train = x_train.copy()
temp_train['hospital_death'] = train_data['hospital_death']

In [36]:
cat_count_var = []

for i,col in tqdm(enumerate(new_cat_var)):
    mean = temp_train['hospital_death'].mean()
    
    #Compute the number of values and the mean of each group
    agg = temp_train.groupby(col)['hospital_death'].agg(['count', 'mean'])
    counts = agg['count']
    means =  agg['mean']
    
    #Compute the "smoothed" means
    m= 3
    smooth = (counts * means + m * mean) / (counts + m)
    
    #Final_val
    x_comb[str(col)+'count_new'] = x_comb[col].map(smooth)
    cat_count_var.append(str(col)+'count_new')
    

32it [00:00, 56.82it/s]


In [37]:
x_comb.shape

(131021, 338)

In [38]:
# # Remove Features with more than 70 percent missing values
data_missing = (x_comb.isnull().sum() / len(x_comb)).sort_values(ascending = False)
data_missing = data_missing.index[data_missing > 0.9]

x_comb = x_comb.drop(columns = data_missing)

In [39]:
cat_count_var_new = list(set(cat_count_var).intersection(x_comb.columns))
#x_comb.isna().sum()

In [40]:
for col in tqdm(cat_count_var_new):
    le = LabelEncoder()
    x_comb[col] = le.fit_transform(x_comb[col].astype(str))

100%|██████████| 21/21 [00:05<00:00,  4.04it/s]


In [41]:
aggs = {}

for col in tqdm(new_cat_var):
    aggs[col] = ['nunique','count']
    

100%|██████████| 32/32 [00:00<00:00, 58584.78it/s]


In [42]:
new_cat_var = list(set(new_cat_var).intersection(set(x_comb.columns)))

In [43]:
#new_cat_var

In [44]:
for col in tqdm(new_cat_var):
    
    aggs_temp = aggs.copy()
    aggs_temp.pop(col)
    
    agg_df = x_comb.groupby(col).agg(aggs_temp).reset_index()
    agg_df.columns = [col] + [col + '_' + c[0] +'_' + c[1] for c in agg_df.columns[1:]]
    
    x_comb = pd.merge(left = x_comb, right= agg_df, how='left',
                    left_on=[col], right_on=[col])

100%|██████████| 32/32 [09:34<00:00, 17.95s/it]


In [45]:
x_comb['hospital_icu_id'] = x_comb['hospital_id'].astype(str) + '_' +  x_comb['icu_id'].astype(str)

le = LabelEncoder()
x_comb['hospital_icu_id'] = le.fit_transform(x_comb['hospital_icu_id'].astype(str))

In [46]:
x_comb = x_comb.drop(['encounter_id','patient_id'],axis=1)

In [47]:
train_x = x_comb[0:91713]
test_x = x_comb[91713:]

In [48]:
x_comb.shape

(131021, 2304)

In [49]:
x1_train, x1_val, y1_train, y1_val = train_test_split(train_x, y_train, 
                                                      test_size=0.2,
                                                      random_state=0,
                                                      stratify = y_train)

In [62]:
random_state = 42
np.random.seed(random_state)

lgb_params = {
    'boost': 'gbdt',
    'learning_rate': 0.01,
    'max_depth':8,
    'metric':{'auc'},
    'num_threads': -1,
    'objective': 'binary',
    'verbosity': 1,
    'num_leaves':16,
    'bagging_fraction':0.5, #rows
    'bagging_frequency':5,
    'min_data_in_leaf':50,
    'feature_fraction':0.22, # features
    'lambda_l1':10,
    'lambda_l2':15,
    'bagging_seed':2019,
    'min_gain_split':0.3
}

In [63]:
trn_data = lgb.Dataset(x1_train, label= y1_train)
val_data = lgb.Dataset(x1_val,  label=  y1_val)

In [64]:
lgb_clf = lgb.train(lgb_params,
                    trn_data,
                    13000,
                    valid_sets = [trn_data, val_data],
                    early_stopping_rounds=2000,
                    verbose_eval=200
                    )

Training until validation scores don't improve for 2000 rounds.
[200]	training's auc: 0.896847	valid_1's auc: 0.885632
[400]	training's auc: 0.906869	valid_1's auc: 0.892779
[600]	training's auc: 0.913636	valid_1's auc: 0.897051
[800]	training's auc: 0.918444	valid_1's auc: 0.899408
[1000]	training's auc: 0.922192	valid_1's auc: 0.900987
[1200]	training's auc: 0.925495	valid_1's auc: 0.902081
[1400]	training's auc: 0.928392	valid_1's auc: 0.902944
[1600]	training's auc: 0.931022	valid_1's auc: 0.90354
[1800]	training's auc: 0.933479	valid_1's auc: 0.904095
[2000]	training's auc: 0.935665	valid_1's auc: 0.904533
[2200]	training's auc: 0.937692	valid_1's auc: 0.904851
[2400]	training's auc: 0.939616	valid_1's auc: 0.905111
[2600]	training's auc: 0.941415	valid_1's auc: 0.905327
[2800]	training's auc: 0.943145	valid_1's auc: 0.905545
[3000]	training's auc: 0.944767	valid_1's auc: 0.905721
[3200]	training's auc: 0.946368	valid_1's auc: 0.905913
[3400]	training's auc: 0.947951	valid_1's auc

KeyboardInterrupt: 

In [ ]:
pred = lgb_clf.predict(test_x)

In [ ]:
data_sub = x_test[['encounter_id']]
data_sub['hospital_death'] = pred

In [ ]:
data_sub.to_csv('/Users/s0c02nj/Desktop/WiDS/sub31_5thFeb2020_binned_labs blood gas.csv',index = False)

In [ ]:
# fig, ax = plt.subplots(figsize=(12,30))
# lgb.plot_importance(lgb_clf, max_num_features=250, height=0.8, ax=ax)
# ax.grid(False)
# plt.title("LightGBM - Feature Importance", fontsize=12)
# plt.show()


In [65]:
kf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)

In [66]:
predictions = np.zeros(len(x_test))
num_models = 0

for train_index, valid_index in tqdm(kf.split(train_x, y_train)):
    
    d_train = lgb.Dataset(train_x.iloc[train_index], label=y_train[train_index])
    d_val = lgb.Dataset(train_x.iloc[valid_index], label=y_train[valid_index])

    clf = lgb.train(lgb_params, d_train, 130000, verbose_eval=1000, 
                    valid_sets = [d_train, d_val], early_stopping_rounds = 3000)
    

    
    num_models += 1
    
    predictions += clf.predict(test_x)

0it [00:00, ?it/s]

Training until validation scores don't improve for 3000 rounds.
[1000]	training's auc: 0.920417	valid_1's auc: 0.902754
[2000]	training's auc: 0.933226	valid_1's auc: 0.906563
[3000]	training's auc: 0.942091	valid_1's auc: 0.908105
[4000]	training's auc: 0.949379	valid_1's auc: 0.908666
[5000]	training's auc: 0.95565	valid_1's auc: 0.908981
[6000]	training's auc: 0.961051	valid_1's auc: 0.909103
[7000]	training's auc: 0.965872	valid_1's auc: 0.909136
[8000]	training's auc: 0.970041	valid_1's auc: 0.909164
[9000]	training's auc: 0.973656	valid_1's auc: 0.909047
[10000]	training's auc: 0.976991	valid_1's auc: 0.908961
Early stopping, best iteration is:
[7672]	training's auc: 0.968739	valid_1's auc: 0.909209


1it [05:51, 351.65s/it]

Training until validation scores don't improve for 3000 rounds.
[1000]	training's auc: 0.919719	valid_1's auc: 0.909166
[2000]	training's auc: 0.932692	valid_1's auc: 0.913696
[3000]	training's auc: 0.941439	valid_1's auc: 0.915035
[4000]	training's auc: 0.948543	valid_1's auc: 0.915442
[5000]	training's auc: 0.954818	valid_1's auc: 0.915607
[6000]	training's auc: 0.960237	valid_1's auc: 0.915833
[7000]	training's auc: 0.965021	valid_1's auc: 0.915728
[8000]	training's auc: 0.969325	valid_1's auc: 0.915539
Early stopping, best iteration is:
[5872]	training's auc: 0.95959	valid_1's auc: 0.915885


2it [10:43, 333.66s/it]

...

8it [45:31, 339.89s/it]

Training until validation scores don't improve for 3000 rounds.
[1000]	training's auc: 0.920399	valid_1's auc: 0.904998
[2000]	training's auc: 0.933391	valid_1's auc: 0.907967
[3000]	training's auc: 0.942153	valid_1's auc: 0.908789
[4000]	training's auc: 0.949503	valid_1's auc: 0.909081
[5000]	training's auc: 0.955666	valid_1's auc: 0.909056
[6000]	training's auc: 0.960991	valid_1's auc: 0.908901
Early stopping, best iteration is:
[3909]	training's auc: 0.948893	valid_1's auc: 0.909102


9it [49:31, 309.98s/it]

Training until validation scores don't improve for 3000 rounds.
[1000]	training's auc: 0.920017	valid_1's auc: 0.906426
[2000]	training's auc: 0.933081	valid_1's auc: 0.909915
[3000]	training's auc: 0.94173	valid_1's auc: 0.910957
[4000]	training's auc: 0.948812	valid_1's auc: 0.911517
[5000]	training's auc: 0.955061	valid_1's auc: 0.91176
[6000]	training's auc: 0.960491	valid_1's auc: 0.911846
[7000]	training's auc: 0.965332	valid_1's auc: 0.91191
[8000]	training's auc: 0.969622	valid_1's auc: 0.911831
[9000]	training's auc: 0.973454	valid_1's auc: 0.911653
[10000]	training's auc: 0.976781	valid_1's auc: 0.911549
Early stopping, best iteration is:
[7106]	training's auc: 0.965792	valid_1's auc: 0.911943


10it [55:01, 330.10s/it]


In [67]:
predictions_10 = predictions / 10.0

In [68]:
data_sub = x_test[['encounter_id']]

In [69]:
data_sub['hospital_death'] = predictions_10

/Users/s0c02nj/anaconda2/envs/graph/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [70]:
data_sub.to_csv('/Users/s0c02nj/Desktop/WiDS/sub_type5_10thFeb2020_withmissing.csv',index = False)

In [ ]:
# a1 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/Submission/sub_ensemble.csv')
# a2 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/Submission/sub_ensemble_gm.csv')
# a3 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/Submission/sub_ensemble_gm1.csv')
# a4 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_ensemble_gm2.csv')
# a5 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub8_lgb_10fold1.csv')
# a6 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub8_lgb_10fold.csv')
# a7 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub10_lgb_kfold2.csv')
# a8 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub5_lgb_ensemble12_fold_01022020.csv')
# a9 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub11_ensemble_imp1.csv')
# a10 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub12_catb.csv')
a11 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub3_lgb_ensemble1_01022020.csv')
a12 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub4_ensemble1_01022020.csv')
a13 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub5_lgb_ensemble12_fold_01022020.csv')
a14 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub3_10fold_4thFeb2020.csv')
a15 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_ensemble_gm3.csv')
a16 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub6_10fold_4thFeb2020.csv')
a17 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub60_10fold_4thFeb2020.csv')
a18 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub2_ensemble_5thFeb2020.csv')
a19 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_type1_type2_ensemble_5thFeb2020.csv')
a20 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub31_5thFeb2020_binned_labs blood gas.csv')
a21 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_type4_6thFeb2020.csv')


In [ ]:
sub = pd.DataFrame()
sub['encounter_id'] = a18['encounter_id']

In [ ]:
pred_ens = (
#             a1['hospital_death'] + a2['hospital_death'] + a3['hospital_death'] + 
#             a4['hospital_death'] + a5['hospital_death'] + a6['hospital_death'] +
#             a7['hospital_death'] + a8['hospital_death'] + a9['hospital_death'] +
#             a10['hospital_death'] + 
            a11['hospital_death'] + a12['hospital_death'] +
            a13['hospital_death'] + a14['hospital_death'] + a15['hospital_death'] +
            a16['hospital_death'] + a17['hospital_death'] + a18['hospital_death'] +
            a19['hospital_death'] + a20['hospital_death'] + a21['hospital_death']
            
           )/11.0
            

In [ ]:
sub['hospital_death'] = pred_ens

In [ ]:
sub.to_csv('/Users/s0c02nj/Desktop/WiDS/sub2_ensemble_6thFeb2020.csv',index = False)

In [ ]:
#train_x['encounter_id'].unique().shape


In [ ]:
#last let us make the submission,note that you have to make the pred to be int!
pred = model.predict_proba(test_x)
preds= pred[:,1]

In [ ]:
sub_data

In [ ]:
data_sub = x_test[['encounter_id']]
data_sub['hospital_death'] = preds

In [ ]:
x_test.head()

In [ ]:
ls

In [ ]:
data_sub.to_csv('/Users/s0c02nj/Desktop/WiDS/sub2.csv',index = False)

In [ ]:
data_sub.shape